In [3]:
import network
import numpy as np

In [4]:
sizes = [4,2,3]
net = network.Network(sizes)

def printValues(listVals):
   for i in range(len(listVals)):
        print(listVals[i])
        print() 

In [184]:
sizes = [4,2,3]
net = network.Network(sizes)

print("original weights: (list of " + str(type(net.weights[0])) + ")")
print(type(net.weights[0]))
printValues(net.weights)
print("shape of net.weights[0]: " + str(net.weights[0].shape))


# manually set weights for connections between layers 0 and 1
net.weights[0]=np.array([[0.1, 0.2, 0.3, 0.4], [1.1,1.2,1.3,1.4]])
# same for layers 1 and 2
net.weights[1]=np.array([[0.5, -0.5], [0.6, -1], [0.7, -2]])

# manually set biases as well
net.biases[0] = np.array([[1.0], [1.0]])
net.biases[1] = np.array([[2.0], [2.0], [2.0]])

print("\nnew weights:")
printValues(net.weights)

print("new biases: (list of " + str(type(net.biases[0])) + ")")
print(type(net.biases[0]))
printValues(net.biases)

original weights: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[0.85717487 0.10822399 0.71144563 0.84662634]
 [0.31944781 0.83490758 0.87195515 0.78300267]]

[[0.9582035  0.5474238 ]
 [0.98809968 0.29596501]
 [0.60158228 0.6476504 ]]

shape of net.weights[0]: (2, 4)

new weights:
[[0.1 0.2 0.3 0.4]
 [1.1 1.2 1.3 1.4]]

[[ 0.5 -0.5]
 [ 0.6 -1. ]
 [ 0.7 -2. ]]

new biases: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[1.]
 [1.]]

[[2.]
 [2.]
 [2.]]



In [166]:
# manually calculate output for given input
# returns list of the activation at each layer
def evaluate(net, cur):
    #print("expected output = " + str(net.feedforward(cur))) # known solution

    # slow way of doing it:
#     for layer in range(net.num_layers-1):
#         tmp = []
#         for node in range(len(net.weights[layer])):
#             w = net.weights[layer][node]
#             b = net.biases[layer][node]
#             print("w = " + str(w))
#             print("b = " + str(b))
#             x = np.dot(w, cur) + b
#             x = network.sigmoid(x)
#             tmp.append([x[0]])
#         cur = tmp
#         print("new cur = " + str(cur) + "\n")

    activations = []
    #print("\n\nfaster way:")
    # fast way of doing it:
    a = cur # input
    activations.append(a)
    for b,w in zip(net.biases, net.weights):
        a = network.sigmoid(np.dot(w,a)+b)
        activations.append(a)
        #print(str(b) + "\n" + str(w) + "\n -> " + str(a) + "\n")
    return activations

In [185]:
# do backpropogation
# (currently just calculates errors at each node)
def backprop(cur, expected):
    activations = evaluate(net, cur)
    print("\nactivations:")
    print(activations)
    
    
    # array to store the calculated error at each node as we backpropogate
    # errors[l][j] = dC/dzl_0
    errors = []
    for n in net.sizes:
        errors.append(np.zeros((n, 1), dtype=float))
    #print('errors:')
    #print(errors)
    # traverse layers backwards
    # TODO: consider adding a dummy layer of to the beginning of the weights and biases vectors
    #       so the indicies make sense and match up with the errors array, etc
    for l in range(len(net.sizes)-1, 0, -1):
        index = l-1 # notes: subtract 1 from layer number to index into weights and biases arrays
        #print("at layer " + str(l))
        # traverse nodes in layer l
        for j in range(net.sizes[l]):
            al_j = activations[l][j]
            # if we're on the last layer
            if l == len(net.sizes)-1:
                errors[l][j] = 2*(al_j-expected[j]) * al_j*(1-al_j) # BP1
            else:
                # BP2
                total = 0
                for k in range(0, net.sizes[l+1]):
                    #print("k = " + str(k) + ", weight -> " + str(net.weights[index+1][k][j]))
                    #print("error -> " + str(errors[l+1][k][0]))
                    total += net.weights[index+1][k][j] * errors[l+1][k][0]
                errors[l][j] = al_j*(1-al_j) + total

            # TODO: as test update the bias and weights corresponding to the error just calculated
            # (not sure if doing this right)
            net.biases[index][j] += -0.25 * errors[l][j]
            if (l != 0):
                for k in range(0, net.sizes[l-1]):
                    net.weights[index][j] += -0.25 * errors[l][j]*activations[l-1][k]
            
    #errors = np.zeros(tuple(net.sizes), dtype=float)
    #print("final errors:")
    #print(errors) # (TODO: hand check these)

In [189]:
# test backpropogation to see if we can get it to work for 2 or 3 training examples

data = np.array([[1], [2], [3], [4]]) # input
expected = np.array([[0.0], [1.0], [0.0]]) # known output for this input

# other training data
data2 = np.array([[-1], [-4], [-99], [0.3]])
expected2 = np.array([[0.333], [0.333], [0.333]])

data3 = np.array([[7.0], [4], [9], [6]])
expected3 = np.array([[0], [0], [1.0]])

for i in range(4000):
    if i%3 ==0:
        backprop(data, expected)
    elif i%3 ==1:
        #backprop(data2, expected2)
        pass
    else:
        backprop(data3, expected3)


activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00989537],
       [0.98591963],
       [0.00146655]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00989442],
       [0.98592234],
       [0.00146655]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00989347],
       [0.98573115],
       [0.00146869]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00989252],
       [0.98573397],
       [0.00146869]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00989157],
       [0.98553772],
       [0.00147084]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00989062],
       [0.98554066],
       [0.00147083]])]

activations:
[array([[1],
       [2],
 


activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00978146],
       [0.94107751],
       [0.00160955]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00978054],
       [0.94125842],
       [0.00160954]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00977963],
       [0.93831395],
       [0.00161212]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00977871],
       [0.93852029],
       [0.00161212]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00977779],
       [0.93532049],
       [0.00161471]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00977688],
       [0.9355568 ],
       [0.0016147 ]])]

activations:
[array([[1],
       [2],
 


activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.0096696 ],
       [0.48712611],
       [0.00178647]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00966871],
       [0.51914726],
       [0.00178646]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00966783],
       [0.4867606 ],
       [0.00178964]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00966694],
       [0.51880361],
       [0.00178963]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00966605],
       [0.48643644],
       [0.00179282]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00966517],
       [0.51849877],
       [0.00179282]])]

activations:
[array([[1],
       [2],
 

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00957952],
       [0.51611271],
       [0.00196389]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00957866],
       [0.48390031],
       [0.00196773]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.0095778 ],
       [0.51611195],
       [0.00196772]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00957694],
       [0.48389959],
       [0.00197157]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00957608],
       [0.51611128],
       [0.00197156]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00957522],
       [0.48389896],
       [0.00197543]])]

activations:
[array([[7.],
       [4.],



activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00946767],
       [0.51610601],
       [0.00225319]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00946683],
       [0.483894  ],
       [0.00225824]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.009466  ],
       [0.51610601],
       [0.00225823]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00946517],
       [0.483894  ],
       [0.0022633 ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00946434],
       [0.51610601],
       [0.00226329]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.0094635 ],
       [0.483894  ],
       [0.00226838]])]

activations:
[array([[7.],
       [4.],

...

       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00835501],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00835444],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00835386],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00835329],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00835272],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00835214],
       [0.516106  ],
       [0.483894  ]])]

activations:
[ar

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00828087],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00828031],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00827975],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00827919],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00827863],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00827808],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],



activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.0082043],
       [0.483894 ],
       [0.516106 ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00820376],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00820322],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00820267],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00820213],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00820159],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
    


activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00813246],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00813193],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.0081314],
       [0.483894 ],
       [0.516106 ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00813087],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00813034],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00812981],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
    


activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00806039],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00805988],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00805936],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00805885],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
       [3],
       [4]]), array([[0.],
       [1.]]), array([[0.00805833],
       [0.483894  ],
       [0.516106  ]])]

activations:
[array([[7.],
       [4.],
       [9.],
       [6.]]), array([[0.],
       [1.]]), array([[0.00805782],
       [0.516106  ],
       [0.483894  ]])]

activations:
[array([[1],
       [2],
 